# Sequential Labelling Lora

## Import Modules

In [ ]:
!pip install -q evaluate nusacrowd seqeval peft
import evaluate
import numpy as np
import transformers
import tensorflow as tf
import pandas as pd
import torch
import wandb

from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorForTokenClassification, create_optimizer, TFAutoModelForTokenClassification, pipeline
from IPython.display import display, HTML
from transformers.keras_callbacks import KerasMetricCallback, PushToHubCallback

In [ ]:
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api_key") 

wandb.login(key=my_secret)
wandb.init(entity = "adiyansawicaksana", project = "huggingface")

## Import Dataset

In [ ]:
nergrit = load_dataset('NusaCrowd/nergrit')

In [ ]:
print(nergrit["train"][0])

In [ ]:
# Extract the 'ner_tag' column from the training set
ner_tags_list = nergrit["train"]["ner_tag"]

# Flatten the list of lists
flattened_ner_tags = [tag for sublist in ner_tags_list for tag in sublist]

# Get the unique labels
unique_labels = list(set(flattened_ner_tags))

print(unique_labels)

Based on the documentation in https://huggingface.co/datasets/NusaCrowd/nergrit.
<br>
Label:

'CRD': Cardinal

'DAT': Date

'EVT': Event

'FAC': Facility

'GPE': Geopolitical Entity

'LAW': Law Entity (such as Undang-Undang)

'LOC': Location

'MON': Money

'NOR': Political Organization

'ORD': Ordinal

'ORG': Organization

'PER': Person

'PRC': Percent

'PRD': Product

'QTY': Quantity

'REG': Religion

'TIM': Time

'WOA': Work of Art

'LAN': Language

In [ ]:
labels_dict = {
    'O': 0,
    'B-CRD': 1, 'I-CRD': 2,
    'B-DAT': 3, 'I-DAT': 4,
    'B-EVT': 5, 'I-EVT': 6,
    'B-FAC': 7, 'I-FAC': 8,
    'B-GPE': 9, 'I-GPE': 10,
    'B-LAW': 11, 'I-LAW': 12,
    'B-LOC': 13, 'I-LOC': 14,
    'B-MON': 15, 'I-MON': 16,
    'B-NOR': 17, 'I-NOR': 18,
    'B-ORD': 19, 'I-ORD': 20,
    'B-ORG': 21, 'I-ORG': 22,
    'B-PER': 23, 'I-PER': 24,
    'B-PRC': 25, 'I-PRC': 26,
    'B-PRD': 27, 'I-PRD': 28,
    'B-QTY': 29, 'I-QTY': 30,
    'B-REG': 31, 'I-REG': 32,
    'B-TIM': 33, 'I-TIM': 34,
    'B-WOA': 35, 'I-WOA': 36,
    'B-LAN': 37, 'I-LAN': 38
}

## Preprocessing

In [ ]:
# use IndoBERT
tokenizer = AutoTokenizer.from_pretrained('indolem/indobert-base-uncased')

In [ ]:
example = nergrit["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

Based on the documentation:

> This adds some special tokens [CLS] and [SEP] and the subword tokenization creates a mismatch between the input and labels. A single word corresponding to a single label may now be split into two subwords. You'll need to realign the tokens and labels by:

1. Mapping all tokens to their corresponding word with the word_ids method.
2. Assigning the label -100 to the special tokens [CLS] and [SEP] so they're ignored by the PyTorch loss function.
3. Only labeling the first token of a given word. Assign -100 to other subtokens from the same word.

So it is needed to realign the token and labels, and truncate the sequence if it is longer than the models maximum length

Based on the reference:

> It's more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

So, creating the data collator.

In [ ]:
# Rename the 'labels' column to 'ner_tag'
nergrit["train"] = nergrit["train"].rename_column("ner_tag", "labels")
nergrit["validation"] = nergrit["validation"].rename_column("ner_tag", "labels")
nergrit["test"] = nergrit["test"].rename_column("ner_tag", "labels")

In [ ]:
def tokenize_and_align_labels(dataset):
    tokenized_inputs = tokenizer(dataset["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, labels_per_example in enumerate(dataset["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                token_labels = [labels_dict[label] for label in labels_per_example]
                label_ids.append(token_labels[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_nergrit = nergrit.map(tokenize_and_align_labels, batched=True) #processing in batch

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

## Metrics Evaluation

I will be using just accuracy

In [ ]:
seqeval = evaluate.load("seqeval")

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [unique_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [unique_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "accuracy": results["overall_accuracy"],
    }

## Fine-Tuning the Model

In [ ]:
labels = list(labels_dict.keys())

id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}


In [ ]:
len(labels)

In [ ]:
batch_size = 16
num_train_epochs = 3
num_train_steps = (len(tokenized_nergrit["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [ ]:
model = TFAutoModelForTokenClassification.from_pretrained(
    "indolem/indobert-base-uncased", 
    num_labels=39,          # set the num labels to 39
    id2label=id2label, 
    label2id=label2id, 
    from_pt=True
)

In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_nergrit["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_nergrit["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
# create the callback for the model
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

push_to_hub_callback = PushToHubCallback(
    output_dir="indobert-base-uncased-finetuned-nergrit",  # change this based on the output repo desired
    tokenizer=tokenizer,
)

wandb_callback = wandb.keras.WandbCallback()

callbacks = [metric_callback, push_to_hub_callback, wandb_callback]

In [ ]:
model.fit(x=tf_train_set, 
          validation_data=tf_validation_set, 
          epochs=25,                 
          callbacks=callbacks)

## Inferencing the Model

In [ ]:
text = """Jakarta, Maret 1998
Di sebuah senja, di sebuah rumah susun di Jakarta, mahasiswa bernama Biru Laut disergap empat lelaki tak dikenal. Bersama kawan-kawannya, Daniel Tumbuan, Sunu Dyantoro, Alex Perazon, dia dibawa ke sebuah tempat yang tak dikenal. Berbulan-bulan mereka disekap, diinterogasi, dipukul, ditendang, digantung, dan disetrum agar bersedia menjawab satu pertanyaan penting: siapakah yang berdiri di balik gerakan aktivis dan mahasiswa saat itu.
Jakarta, Juni 1998
Keluarga Arya Wibisono, seperti biasa, pada hari Minggu sore memasak bersama, menyediakan makanan kesukaan Biru Laut. Sang ayah akan meletakkan satu piring untuk dirinya, satu piring untuk sang ibu, satu piring untuk Biru Laut, dan satu piring untuk si bungsu Asmara Jati. Mereka duduk menanti dan menanti. Tapi Biru Laut tak kunjung muncul.
Jakarta, 2000
Asmara Jati, adik Biru Laut, beserta Tim Komisi Orang Hilang yang dipimpin Aswin Pradana mencoba mencari jejak mereka yang hilang serta merekam dan mempelajari testimoni mereka yang kembali. Anjani, kekasih Laut, para orangtua dan istri aktivis yang hilang menuntut kejelasan tentang anggota keluarga mereka. Sementara Biru Laut, dari dasar laut yang sunyi bercerita kepada kita, kepada dunia tentang apa yang terjadi pada dirinya dan kawan-kawannya.
Laut Bercerita, novel terbaru Leila S. Chudori, bertutur tentang kisah keluarga yang kehilangan, sekumpulan sahabat yang merasakan kekosongan di dada, sekelompok orang yang gemar menyiksa dan lancar berkhianat, sejumlah keluarga yang mencari kejelasan akan anaknya, dan tentang cinta yang tak akan luntur."""

In [ ]:
# use pipeline to easen inferencing the model
classifier = pipeline("ner", model="apwic/indobert-base-uncased-finetuned-nergrit")

In [ ]:
classified_text = classifier(text)

In [ ]:
pd.DataFrame(classified_text[:10])

In [ ]:
def visualize_entities_html(text, classified_text):
    # Create a color mapping for each entity type
    color_map = {
        'GPE': 'yellow',
        'DAT': 'blue',
        'PER': 'green',
        'CRD': 'red',
        'EVT': '#FFA500',  # orange
        'FAC': '#FFC0CB',  # pink
        'LAW': '#FFD700',  # gold
        'LOC': '#ADFF2F',  # greenyellow
        'MON': '#FA8072',  # salmon
        'NOR': '#9370DB',  # mediumpurple
        'ORD': '#7B68EE',  # mediumslateblue
        'ORG': '#6A5ACD',  # slateblue
        'PRC': '#FF69B4',  # hotpink
        'PRD': '#D2B48C',  # tan
        'QTY': '#FF6347',  # tomato
        'REG': '#DB7093',  # palevioletred
        'TIM': '#EEE8AA',  # palegoldenrod
        'WOA': '#F08080',  # lightcoral
        'LAN': '#BDB76B'   # darkkhaki
    }

    
    # Sort classified_text by start index
    classified_text.sort(key=lambda x: x['start'])

    html_output = text
    shift = 0
    
    for entity in classified_text:
        word = entity['word']
        start = entity['start'] + shift
        end = entity['end'] + shift
        entity_type = entity['entity'].split('-')[-1]  # Extracting main entity type e.g., 'B-GPE' -> 'GPE'
        color = color_map.get(entity_type, 'grey')  # Default to grey if entity type is not in our map
        
        # Wrap the word in a span with background color
        span = f"<span style='background-color: {color}'>{word}</span>"
        
        # Replace the word in the text with the highlighted word
        html_output = html_output[:start] + span + html_output[end:]
        
        # Adjust shift based on the added HTML tags
        shift += len(span) - (end - start)
    
    display(HTML(html_output))

visualize_entities_html(text, classified_text)